In [182]:
import math
from datetime import datetime, date, timedelta
import random
import matplotlib.pyplot as plt
from statistics import mean
import imp
import os
import numpy as np
from pulp import *
import warnings

warnings.filterwarnings('ignore')

# Solve LP

In [190]:
# Solve
def solveLP(prob, LHS, RHS, W, pvt, T, K, lambd):
    # LHS (U): server nodes, RHS (V): call type nodes, W: weights
    # pvt: probability of v coming at time t
    # T: time horizons, K: occupied time steps
    # lambd: parameter for exponential distribution
    X = [[[LpVariable("edge_{}_{}_{}".format(str(u + 1), str(v + 1), str(t + 1)), lowBound=0.0, upBound=1.0,
                      cat='Continuous') for v in RHS] for u in LHS] for t in range(T)]
    print(np.shape(X))
    prob += np.sum((W[t][u][v] * X[t][u][v] for v in RHS for u in LHS for t in range(T)))

    # p = xp.problem("offline linear program")
    # p.addVariable(X)

    # constraint 1: no customer over matched
    for t in range(T):
        for v in RHS:
            prob += np.sum(X[t][u][v] for u in LHS) <= pvt[t][v]

    # constraint 2: no server over matched
    for t in range(T):
        for u in LHS:
            prob += np.sum(
                X[tau][u][v] * math.exp(-lambd * (t - tau)) for v in RHS for tau in range(t - K, t)) + np.sum(
                X[t][u][v] for v in RHS) <= 1

    prob.solve()

    X_unnested = prob.variables()
    _next = 0

    for u in LHS:
        for v in RHS:
            for t in range(T):
                X[t][u][v] = X_unnested[_next].varValue
                _next += 1

    return X, prob.objective.value()
    # return [v.varValue for v in prob.variables()], prob.objective.value()

### test the offline optimal matching through solving LP

In [199]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("matching", LpMaximize)
# test the offline optimal
U, V = 10, 4
T, K = 10, 2
lambd = 0.5
W = [[[random.uniform(0, 1) for j in range(V)] for i in range(U)] for t in range(T)]
pvt = [[1. / V for v in range(V)] for t in range(T)]
LHS, RHS = range(U), range(V)

In [200]:
Xopt, optimal = solveLP(prob, LHS, RHS, W, pvt, T, K, lambd)

(100, 50, 4)
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/haolunwu/opt/anaconda3/envs/py38/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/0t/55spm7k95h33phsgfwny8m600000gn/T/84879bb76c094f528132876b67ec40db-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/0t/55spm7k95h33phsgfwny8m600000gn/T/84879bb76c094f528132876b67ec40db-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5405 COLUMNS
At line 105406 RHS
At line 110807 BOUNDS
At line 130808 ENDATA
Problem MODEL has 5400 rows, 20000 columns and 80000 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 5400 (0) rows, 20000 (0) columns and 80000 (0) elements
0  Obj -0 Dual inf 10057.804 (20000)
183  Obj 2750.6916 Primal inf 11210.246 (4712)
366  Obj 1784.3742 Primal inf 6533.0232 (3027)
549  Obj 444.59092 Primal inf 993.09903 (672)
732  Obj 99.987

In [186]:
Xopt, optimal

([[[0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.25, 0.0, 0.25],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.25, 0.0],
   [0.25, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0]],
  [[0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.25, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.25, 0.25, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.25],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0]],
  [[0.0, 0.0, 0.0, 0.0],
   [0.0, 0.25, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.25],
   [0.0, 0.0, 0.25, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.25, 0.0, 0.0, 0.0]],
  [[0.0, 0.0, 0.25, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.25, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.25],
   [0.25, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.

In [188]:
np.shape(Xopt)

(5, 10, 4)